In [ ]:
import micromegas
import datetime
import pandas as pd
pd.set_option('display.max_colwidth', None)
client = micromegas.connect()

In [18]:
%%time
sql = """
SELECT view_set_name, view_instance_id, min(begin_insert_time) as begin, max(end_insert_time) as end
FROM list_partitions()
where view_instance_id != 'global'
group by view_set_name, view_instance_id
"""
df_instances = client.query(sql)
for _, row in df_instances.iterrows():
    client.retire_partitions(row["view_set_name"], row["view_instance_id"], row["begin"], row["end"])

2025-08-28 18:17:52.573016276+00:00 adding out of date partition views/async_events/f126ae57-7066-4f68-9378-235079bbb433/2025-08-28/17-47-29_34671ddf-553b-4d2f-9d8d-5b53a02cedfe.parquet to temporary files to be deleted
2025-08-28 18:17:52.573353871+00:00 adding out of date partition views/async_events/f126ae57-7066-4f68-9378-235079bbb433/2025-08-28/17-47-29_d950cc9a-24dd-4ce7-b19d-e57b734396ec.parquet to temporary files to be deleted
2025-08-28 18:17:52.573572661+00:00 adding out of date partition views/async_events/f126ae57-7066-4f68-9378-235079bbb433/2025-08-28/17-47-29_6dad303d-7d7d-4ce1-97a5-0888dc688b49.parquet to temporary files to be deleted
2025-08-28 18:17:52.573825019+00:00 adding out of date partition views/async_events/f126ae57-7066-4f68-9378-235079bbb433/2025-08-28/17-47-29_671efa0d-74fe-423e-883b-5031cfb46243.parquet to temporary files to be deleted
2025-08-28 18:17:52.574020625+00:00 adding out of date partition views/async_events/f126ae57-7066-4f68-9378-235079bbb433/202

In [ ]:
%%time
now = datetime.datetime.now(datetime.timezone.utc)
begin = now - datetime.timedelta(days=10)
begin = datetime.datetime.combine(begin, datetime.time(), tzinfo=begin.tzinfo)
end = datetime.datetime.combine(now, datetime.time(), tzinfo=now.tzinfo)
client.retire_partitions('async_events', 'd35a4b40-7407-40cd-a1cf-c619124ee529', begin, end)

In [ ]:
%%time
sql = "SELECT * FROM processes WHERE process_id='d35a4b40-7407-40cd-a1cf-c619124ee529'"
df = client.query(sql)
df["tsc_frequency"]

In [ ]:
%%time
#backfill day-long partitions
now = datetime.datetime.now(datetime.timezone.utc)
begin = now - datetime.timedelta(days=10)
begin = datetime.datetime.combine(begin, datetime.time(), tzinfo=begin.tzinfo)
end = datetime.datetime.combine(now, datetime.time(), tzinfo=now.tzinfo)
size = 60*60*24
client.materialize_partitions( "blocks", begin, end, size )
client.materialize_partitions( "streams", begin, end, size )
client.materialize_partitions( "processes", begin, end, size )